# Logistic Regression From Scratch

In [53]:
from typing import Any

import numpy as np
from numpy.typing import NDArray

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

In [54]:
raw_x, raw_y = make_classification(n_features=10,n_samples=1000, random_state=3442)

In [55]:
raw_x.shape, raw_y.shape

((1000, 10), (1000,))

In [56]:
raw_x[:5]

array([[-0.56573764, -1.12659397, -1.58488824,  0.047902  ,  0.60861322,
        -1.23263319, -0.36952168, -0.5890378 ,  0.12181396, -1.57697136],
       [ 0.00678233, -0.84560635, -0.41794588, -1.02328482, -0.51747583,
         0.1463818 ,  0.78425481,  0.07079281,  0.0927115 ,  0.24143793],
       [ 0.75739895,  1.21942221,  1.37968123, -0.3908939 , -0.88958722,
         0.86788148, -0.26789949, -0.40195265,  1.06290301,  1.08675515],
       [-1.14172981,  1.48598207,  0.70375216,  0.62227436,  1.39281664,
        -0.29987348,  0.58863519, -0.83229884,  0.40897603, -0.48098127],
       [ 1.84484214, -1.11423938, -0.12115165, -1.31060452, -0.4074719 ,
         0.78941927, -0.59384031, -0.86682628,  0.70957145,  1.11146838]])

In [57]:
raw_y.shape[0]

1000

In [58]:
oe = OneHotEncoder()
value = oe.fit_transform(raw_y.reshape(-1,1))
encoded_y = value.toarray()
encoded_y

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [59]:
x_train, x_test, y_train, y_test = train_test_split(raw_x, raw_y, random_state=3453, test_size=.2)
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((800, 10), (800,)), ((200, 10), (200,)))

In [60]:
class LogisticRegression:
    """implement logistic regression from scratch"""

    def __init__(self, learning_rate: float = 0.001, epochs: int = 500, classes: int = 2) -> None:
        """initializing model & hyper parameters"""
        
        self.learning_rate = learning_rate
        self.epochs = epochs  
        self.no_of_classes = classes
        self.bias = None 
        self.weight= None

    def linear_equation(self, x: NDArray[np.float64], w: NDArray[np.float64], b: float | NDArray) -> NDArray:
        """
            equation: z = w . x + b
        """
        if x.shape[1] != w.shape[0]:
            raise ValueError("X and W are mismatched column count")

        return np.dot(x, w) + b
        
    @staticmethod
    def sigmoid(z: NDArray) -> NDArray:
        """
            equation: sigmoid(z) = 1 / 1 + e ^ -z
        """
        result = [1/(1+np.exp(-each)) for each in z]
        return np.array(result)


    @staticmethod
    def softmax(z: NDArray[float]) -> NDArray:
        """
            equation: softmax(z) = e^z / sum(e^z)
        """
        sum_of_exp = sum([np.exp(each) for each in z])
        
        result = [np.round(np.exp(each)/sum_of_exp, 5) for each in z]
        return np.array(result)

    
    def get_dw(self, y_true: NDArray[int], y_prob: NDArray[float], x: NDArray[float]) -> float:
        """generate partial derivative of loss with respect to weight
        
            Equation:
                dw = 1/n . y_prob_i - y_true_i . x_i
        """
        if self.no_of_classes > 2:
            result = []
            for yp_i, y_i, x_i in zip(y_prob, y_true, x):
                error = np.array(yp_i) - np.array(y_i)
                result.append(np.outer(error, x_i))
            sum_of_grad = sum(result)
            dw = np.mean(sum_of_grad)
            return np.array(dw)
        
        error = y_prob - y_true
        dw = []
        for each in x.T:
            result = np.mean([np.round(a*b, 3) for a, b in zip(error, each)])
            dw.append(result)
        return np.array(dw)

    def get_db(self, y_true: NDArray[int], y_prob: NDArray[float]) -> float:
        """generate partial derivative of loss with respect to bias"""

        if self.no_of_classes > 2:
            result = []
            for yp_i, y_i in zip(y_prob, y_true):
                error = np.array(yp_i) - np.array(y_i)
                result.append(error)
            sum_of_grad = sum(result)
            db = np.mean(sum_of_grad)
            return np.array(db)
            
        error = y_prob - y_true
        return np.mean(error)
        
    def update_params(self, y_prob: NDArray, y_true: NDArray, x: NDArray) -> tuple[float, float]:
        """
        Returns:
            tuple[float, float] : (weight, bias)
        """
        new_weight = self.weight - (self.learning_rate * self.get_dw(y_prob=y_prob, y_true=y_true, x=x))
        new_bias = self.bias - (self.learning_rate * self.get_db(y_prob=y_prob, y_true=y_true))
        return (new_weight, new_bias)


    def forward_prop(self, x: NDArray) -> Any:

        logits = self.linear_equation(x=x, w=self.weight, b=self.bias)
        return self.softmax(logits) if self.no_of_classes > 2 else self.sigmoid(logits)

    def backward_prop(self, y_prob: NDArray, y_true: NDArray, x: NDArray) -> Any:
        self.weight, self.bias = self.update_params(y_prob=y_prob, y_true=y_true, x=x)

    def fit(self, x: NDArray[np.float64], y: NDArray[np.int16]) -> Any:

        self.weight = np.random.randn(x.shape[1]) * .01 if self.no_of_classes <= 2 else np.random.randn(x.shape[1], self.no_of_classes)
        self.bias = 0 if self.no_of_classes <= 2 else np.zeros(self.no_of_classes)

        for _ in range(self.epochs):
            prob = self.forward_prop(x=x)
            self.backward_prop(x=x, y_prob=prob, y_true=y)

        print("training completed")

    def predict(self, x: np.ndarray) -> Any:
        pred_logits = self.linear_equation(x=x, w=self.weight, b=self.bias)
        
        if self.no_of_classes > 2:
            pred_prob = self.softmax(pred_logits) 
            # result = list(map(lambda x: 1 if x > .5 else 0, pred_prob))
            return np.array(pred_prob)
            
        pred_prob = self.sigmoid(pred_logits)
        result = list(map(lambda x: 1 if x > .5 else 0, pred_prob))
        return np.array(result)

In [61]:
model = LogisticRegression()

model.fit(x=x_train, y=y_train)

training completed


In [62]:
model.weight

array([ 0.01066313,  0.22350471,  0.14785305,  0.01005669, -0.00246937,
        0.01232307,  0.00239741,  0.00210378,  0.03042633, -0.01037553])

In [63]:
model.bias

np.float64(0.0047347655773623005)

In [64]:
y_pred = model.predict(x=x_test)
y_pred

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0])

In [65]:
y_test

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0])

In [66]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(f"TN : {tn}\nFP : {fp}\nFN : {fn}\nTP : {tp}")

TN : 103
FP : 3
FN : 11
TP : 83


In [67]:
test_acc = accuracy_score(y_test, y_pred) * 100
print(f"Test Accuracy is {test_acc}")

Test Accuracy is 93.0


In [68]:
y_pred_train = model.predict(x_train)
train_acc = accuracy_score(y_train, y_pred_train) * 100
print(f"Train Accuracy is {train_acc}")

Train Accuracy is 91.125
